In [1]:
import torch
import numpy as np
from torchvision.transforms import transforms
import random
from PIL import Image
from resnet18 import ResNet18
from torch.autograd import Variable
from resnet18_inc_v2 import IncrementalResNet18V2
import torch.utils.model_zoo as model_zoo

from cnn.imagenet_classes import class_names
from inception3 import Inception3

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from inception3_inc_v2 import IncrementalInception3V2

In [3]:
# batch_size = 1

# loader = transforms.Compose([transforms.Resize([299, 299]), transforms.ToTensor()])
# images = Image.open('./dog_resized.jpg')

# images = loader(images)
# images = images.unsqueeze(0).cuda()

# images = images.repeat(batch_size, 1, 1, 1)

# model = Inception3().cuda()
# model.eval()

# x = model(images)
# print(class_names[np.argmax(x.data.cpu().numpy()[0, :])])

In [4]:
batch_size = 1
patch_size = 16
input_size = 299

image_patch = torch.cuda.FloatTensor(3, patch_size, patch_size).fill_(0)

x_loc = random.sample(range(0, input_size - patch_size), batch_size)
y_loc = random.sample(range(0, input_size - patch_size), batch_size)
patch_locations = zip(x_loc, y_loc)
patch_locations = [(0, 0)]

loader = transforms.Compose([transforms.Resize([299, 299]), transforms.ToTensor()])
images = Image.open('./dog_resized.jpg')
images = loader(images)

images = images.unsqueeze(0)
images = images.repeat(batch_size, 1, 1, 1)

inc_model = IncrementalInception3V2(images, beta=1.0)

for i,(x,y) in enumerate(patch_locations):
    images[i, :, x:x+patch_size, y:y+patch_size] = image_patch

full_model = Inception3()
full_model.eval()
y = full_model.forward_materialized(images.cuda())

patch_locations = torch.from_numpy(np.array(patch_locations, dtype=np.int32))

inc_model.eval()
x = inc_model(images, patch_locations, patch_size, patch_size)
# print(class_names[np.argmax(x.data.cpu().numpy()[0, :])])

temp = (y - x).cpu().data.numpy()
# import matplotlib.pyplot as plt
# plt.imshow(temp[0,1,:,:])
# plt.colorbar()
# plt.show()
print(np.max(np.abs(temp)))

1.66893e-06


In [5]:
np.where(np.abs(temp) > 1e-3)[0].shape

(0,)

In [6]:
temp = (y).cpu().data.numpy()
import matplotlib.pyplot as plt
plt.imshow(temp[0,1,:,:])
plt.colorbar()
plt.show()
print(np.max(np.abs(temp)))

IndexError: too many indices for array

In [ ]:
temp = (x).cpu().data.numpy()
import matplotlib.pyplot as plt
plt.imshow(temp[0,1,:,:])
plt.colorbar()
plt.show()
print(np.max(np.abs(temp)))

In [ ]:
temp = (y - x).cpu().data.numpy()
import matplotlib.pyplot as plt
plt.imshow(temp[0,1,:,:])
plt.colorbar()
plt.show()
print(np.max(np.abs(temp)))